In [17]:
using JuMP, GLPK, Test
const MOI = JuMP.MathOptInterface

model = JuMP.Model(JuMP.with_optimizer(GLPK.Optimizer))
m=11
Ctest=[[0 29 20 21 16 31 100 12 4 31 18];[29 0 15 29 28 40 72 21 29 41 12];
    [20 15 0 15 14 25 81 9 23 27 13];[21 29 15 0 4 12 92 12 25 13 25];
    [16 28 14 4 0 16 94 9 20 16 22];[31 40 25 12 16 0 95 24 36 3 37];
    [100 72 81 92 94 95 0 90 101 99 84];[12 21 9 12 9 24 90 0 15 25 13];
    [4 29 23 25 20 36 101 15 0 35 18];
    [31 41 27 13 16 3 99 25 35 0 38];[18 12 13 25 22 37 84 13 18 38 0]]
#C = rand(m,m)

JuMP.@variable(model, x[1:m,1:m], Bin)
# Objective: minimize total distance
JuMP.@objective(model, Min, sum(Ctest' .* x))
# Constraint: one to one

for i in 1:m
    JuMP.@constraint(model,sum(x[i, :]) == 1)
    JuMP.@constraint(model,sum(x[:, i]) == 1)
    JuMP.@constraint(model,x[i, i] == 0)
end

# Constraint: graph conectivity 2x2 not necesary but boosts efficiency
for i in 1:m
    for j in 1:m
        if i != j 
            JuMP.@constraint(model, x[i,j] + x[j,i] <= 1)
        end

    end
end
    
   status = JuMP.optimize!(model)

# Constraint: eliminate individual cycles
function solved(model,m)
    cycle = [1]
    check = false
    i = 1
    while !check
        for j in 1:m
            if JuMP.value(x[i,j]) == 1
                for k in 1:length(cycle)
                    # comment next line to skip evolution print 1
                    print(cycle[k], " ")
                    if cycle[k] == j
                        check = true
                    end
                end
                if check == false
                    push!(cycle, 0)
                    cycle[length(cycle)] = j
                end
                i = j
                break
            end
        end
        # comment next line to skip evolution print 2
        println()
    end
    if length(cycle) < m
        JuMP.@constraint(model, sum(x[cycle,cycle]) <= length(cycle) - 1)
    else
        return true
    end
        
    return false
end 
    
   #solved(model,m)

   while !solved(model,m)
        status = JuMP.optimize!(model)
   end
    
    println("Objective is: ", JuMP.objective_value(model))
    println("Solution is:")
    for i in 1:m 
        println()
        for j in 1:m
            print(JuMP.value(x[i,j]), " ")
        end
    end
 

#model

1 
1 9 
1 9 11 
1 9 11 2 
1 9 11 2 7 
1 9 11 2 7 3 
1 9 11 2 7 3 8 
1 
1 9 
1 9 11 
1 9 11 2 
1 9 11 2 7 
1 9 11 2 7 3 
1 9 11 2 7 3 8 
1 9 11 2 7 3 8 5 
1 
1 8 
1 8 5 
1 8 5 4 
1 8 5 4 10 
1 8 5 4 10 6 
1 8 5 4 10 6 3 
1 8 5 4 10 6 3 7 
1 8 5 4 10 6 3 7 2 
1 8 5 4 10 6 3 7 2 11 
1 8 5 4 10 6 3 7 2 11 9 
Objective is: 253.0
Solution is:

0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 
0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 
0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 
1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 

In [29]:
Ctest

11×11 Array{Int64,2}:
   0  29  20  21  16  31  100  12    4  31  18
  29   0  15  29  28  40   72  21   29  41  12
  20  15   0  15  14  25   81   9   23  27  13
  21  29  15   0   4  12   92  12   25  13  25
  16  28  14   4   0  16   94   9   20  16  22
  31  40  25  12  16   0   95  24   36   3  37
 100  72  81  92  94  95    0  90  101  99  84
  12  21   9  12   9  24   90   0   15  25  13
   4  29  23  25  20  36  101  15    0  35  18
  31  41  27  13  16   3   99  25   35   0  38
  18  12  13  25  22  37   84  13   18  38   0